In [1]:
import sklearn
sklearn.__version__

'0.18.1'

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns
# отключим предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

## Сделаем функцию, которая будет заменять NaN значения на медиану в каждом столбце таблицы 
def delete_nan(table):
    for col in table.columns:
        table[col]= table[col].fillna(table[col].median())
    return table

## Считываем данные
data = pd.read_csv('../../data/credit_scoring_sample.csv', sep =';')

independent_columns_names = data.columns.values
independent_columns_names = [x for x in data if x != 'SeriousDlqin2yrs']
independent_columns_names

## Применяем функцию, заменяющую все NaN значения на медианное значение соответствующего столбца
table =delete_nan(data)

## Разделяем таргет и признаки 
X =table[independent_columns_names]
y = table['SeriousDlqin2yrs']

from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression

parameters = {'max_features': [2, 3, 4], 'max_samples': [0.5, 0.7, 0.9], "base_estimator__C": [0.0001, 0.001, 0.01, 1, 10, 100]}

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)
lr = LogisticRegression(random_state=5, class_weight= 'balanced')
bg = BaggingClassifier(base_estimator=lr, random_state=42, n_estimators=100)
random_search = RandomizedSearchCV(estimator=bg, param_distributions=parameters, cv=skf, random_state=1, n_iter=20, scoring='roc_auc', n_jobs = -1)

random_search.fit(X, y)

print(random_search)
print(random_search.best_score_)